<a href="https://colab.research.google.com/github/LuanLucasFS/100DaysOfPy/blob/main/BrainTumorIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import kagglehub
import shutil

# Download latest version
path = kagglehub.dataset_download("navoneel/brain-mri-images-for-brain-tumor-detection")

other_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)
print("Other Path to dataset files:", other_path)

glioma_dir = other_path + "/Training/glioma"
meningioma_dir = other_path + "/Training/meningioma"
pituitary_dir = other_path + "/Training/pituitary"
notumor_dir = other_path + "/Training/notumor"


In [ ]:
tumor_dir = path + "/yes"
no_tumor_dir = path + "/no"

shutil.copytree(glioma_dir, tumor_dir, dirs_exist_ok=True)
shutil.copytree(meningioma_dir, tumor_dir, dirs_exist_ok=True)
shutil.copytree(pituitary_dir, tumor_dir, dirs_exist_ok=True)
shutil.copytree(notumor_dir, no_tumor_dir, dirs_exist_ok=True)



In [ ]:
num_tumor_images = len(os.listdir(tumor_dir))
num_no_tumor_images = len(os.listdir(no_tumor_dir))

print("Number of Tumor Images:", num_tumor_images)
print("Number of No Tumor Images:", num_no_tumor_images)

In [ ]:
def show_images(folder, title, num_images=4):
    images = random.sample(os.listdir(folder), num_images)
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

    for i, img_name in enumerate(images):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img)
        axes[i].axis("off")
        axes[i].set_title(title)

    plt.show()

In [ ]:
show_images(tumor_dir, "Tumor MRI")
show_images(no_tumor_dir, "No Tumor MRI")

In [ ]:
IMG_SIZE = 128

def load_and_preprocess_images(folder, label):
    images = []
    labels = []

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(label)

    return images, labels

tumor_images, tumor_labels = load_and_preprocess_images(tumor_dir, 1)
no_tumor_images, no_tumor_labels = load_and_preprocess_images(no_tumor_dir, 0)

X = tumor_images + no_tumor_images
y = tumor_labels + no_tumor_labels

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

X = X / 255.0
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

In [ ]:
other_path = '/root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Testing/'
# Definição das classes e seus respectivos rótulos
classes = {
    "glioma": 1,
    "pituitary": 1,
    "meningioma": 1,
    "notumor": 0
}

for class_name, label in classes.items():
    class_path = os.path.join(other_path, class_name)
    image_files = os.listdir(class_path)
    print(f"Existem {len(image_files)} na pasta {class_name}")

# Número de imagens por classe
num_images = 300

# Lista para armazenar os dados (imagens e labels)
dataset = []

# Percorre cada classe no dicionário
for class_name, label in classes.items():
    class_path = os.path.join(other_path, class_name)  # Caminho da classe
    image_files = os.listdir(class_path)  # Lista de arquivos na pasta

    # Seleciona aleatoriamente `num_images` arquivos
    selected_files = random.sample(image_files, min(num_images, len(image_files)))

    for file in selected_files:
        img_path = os.path.join(class_path, file)  # Caminho completo da imagem
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Lê a imagem com OpenCV

        if img is not None:
            img = cv2.resize(img, (128, 128))  # Redimensiona para 128x128 (pode ajustar)
            dataset.append((img, label))  # Adiciona a imagem e o rótulo

# Embaralha o dataset
random.shuffle(dataset)

# Separar em arrays de imagens e labels
X = np.array([data[0] for data in dataset])  # Imagens
y = np.array([data[1] for data in dataset])  # Labels

print("Total de imagens carregadas:", X.shape)
print("Total de labels carregados:", y.shape)

In [ ]:
# Normaliza as imagens de teste
X_test_analysis = X / 255.0
y_test_analysis = to_categorical(y)

# Faz a predição com o modelo treinado
y_pred = model.predict(X_test_analysis)

# Converte as predições e os labels reais para valores binários (0 ou 1)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_analysis, axis=1)

# Contadores de acertos e erros
acertos = 0
erros = 0

# Define o número de imagens a serem analisadas
num_images_to_show = len(X_test_analysis)

# Exibe as imagens e as previsões
for i in range(num_images_to_show):
    real = y_true_labels[i]
    previsto = y_pred_labels[i]

    # Define cor do texto
    if real == previsto:
        acertos += 1
    else:
        erros += 1

    # Mostra a imagem
    plt.imshow(X_test_analysis[i].reshape(128, 128), cmap='gray')  # Ajuste o tamanho conforme necessário
    plt.title(f"Real: {real} | Previsto: {previsto}")  # Define a cor e reseta no final
    plt.axis('off')
    plt.show()

# Mostra o relatório de classificação
print("\n" + classification_report(y_true_labels, y_pred_labels))

# Exibe a acurácia do modelo
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f"\nAcurácia: {accuracy:.4f}")

# Exibe a contagem de acertos e erros
print(f"\033[92mAcertos: {acertos}\033[0m")  # Verde
print(f"\033[91mErros: {erros}\033[0m")  # Vermelho